In [1]:
import d2lzh as d2l
from mxnet import gluon, init, nd
from mxnet.gluon import nn

class Inception(nn.Block):
    def __init__(self,c1, c2, c3, c4, **kwargs):
        super(Inception, self).__init__(**kwargs)
        # 线路1
        self.p1_1 = nn.Conv2D(c1, kernel_size=1, activation='relu')
        # 线路2
        self.p2_1 = nn.Conv2D(c2[0], kernel_size=1, activation='relu')
        self.p2_2 = nn.Conv2D(c2[1], kernel_size=3, padding=1, activation='relu')
        # 线路3
        self.p3_1 = nn.Conv2D(c3[0], kernel_size=1, activation='relu')
        self.p3_2 = nn.Conv2D(c3[1], kernel_size=5, padding=2, activation='relu')
        # 线路4
        self.p4_1 = nn.MaxPool2D(pool_size=3, strides=1, padding=1)
        self.p4_2 = nn.Conv2D(c4, kernel_size=1, activation='relu')

    def forward(self, x):
        p1 = self.p1_1(x)
        p2 = self.p2_2(self.p2_1(x))
        p3 = self.p3_2(self.p3_1(x))
        p4 = self.p4_2(self.p4_1(x))
        return nd.concat(p1, p2, p3, p4, dim=1) # 在通道维上输出

In [2]:
# 构建GoogLeNet
# 5 个模块 每个模块之间使用步幅为2的3*3最大池化层减小输出高宽
# 第一模块使用64通道的7*7卷积层
b1 = nn.Sequential()
b1.add(nn.Conv2D(32, kernel_size=7, strides=2, padding=3, activation='relu'),
       nn.MaxPool2D(pool_size=3, strides=2, padding=1))

In [3]:
# 第二个模块使用两个卷积层， 首先是64通道的1*1卷积层， 然后是通道增大3倍的3*3卷积层 输出 192
b2 = nn.Sequential()
b2.add(nn.Conv2D(32, kernel_size=1 ,activation='relu'),
       nn.Conv2D(96, kernel_size=3, padding=1, activation='relu'),
       nn.MaxPool2D(pool_size=3, strides=2, padding=1))

In [4]:
# 第三个模块 2个完整的Inception块， 输出通道数为 64+128+32+32 = 256 128+192+96+64=480
# 2:4:1:1 1/2 1/12
# 4:6:3:2 1/2 1/8
b3 = nn.Sequential()
b3.add(Inception(32, (48, 64), (8, 16), 16),
       Inception(64,(64, 96), (16, 48), 32),
       nn.MaxPool2D(pool_size=3, strides=2, padding=1))

In [5]:
# 第四个模块， 5个完整的Inception快
b4 = nn.Sequential()
b4.add(Inception(96, (48,64), (8,24), 48),
       Inception(80, (56,112), (12,24), 32),
       Inception(64, (64,128), (12,32), 32),
       Inception(56, (72,144), (16,32), 32),
       Inception(128, (80,160), (16,64), 64),
       nn.MaxPool2D(pool_size=3, strides=2, padding=1))


In [6]:
b5 = nn.Sequential()
b5.add(Inception(128, (80, 160), (16, 64), 64),
       Inception(192, (96, 192), (24, 64), 64),
       nn.GlobalAvgPool2D())
net = nn.Sequential()
net.add(b1, b2, b3, b4, b5, nn.Dense(10))

In [7]:
X = nd.random.uniform(shape=(1, 1, 96, 96))
net.initialize()
for layer in net:
    X=layer(X)
    print(layer.name, 'output shape :\t ', X.shape)

sequential0 output shape :	  (1, 32, 24, 24)
sequential1 output shape :	  (1, 96, 12, 12)
sequential2 output shape :	  (1, 240, 6, 6)
sequential3 output shape :	  (1, 416, 3, 3)
sequential4 output shape :	  (1, 512, 1, 1)
dense0 output shape :	  (1, 10)


In [8]:
lr, num_epochs, batch_size, ctx = 0.1, 5, 128, d2l.try_gpu()
net.initialize(force_reinit=True, ctx=ctx, init=init.Xavier())
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': lr})
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size, resize=224)
d2l.train_ch5(net, train_iter, test_iter, batch_size, trainer, ctx, num_epochs)

training on gpu(0)
epoch 1, loss 1.8813, train acc 0.288, test acc 0.583, time 194.0 sec
epoch 2, loss 0.8816, train acc 0.649, test acc 0.679, time 189.2 sec
epoch 3, loss 0.5581, train acc 0.791, test acc 0.824, time 189.5 sec
epoch 4, loss 0.4470, train acc 0.834, test acc 0.825, time 191.4 sec
epoch 5, loss 0.4055, train acc 0.850, test acc 0.847, time 189.9 sec
